# Run macro file 

This notebook shows how load an ImageJ macrofile linked to a dataset and analyze all the images contained in that dataset with the macro.
This notebook assumes that a macro file has been linked to the dataset. If more than one macro is linked, the first one found will be applied.

Fiji has been installed with few other plugins. For more information, check the Dockerfile.

In [1]:
//Add dependencies to the classpath
%classpath add jar /opt/java-apps/Fiji.app/jars/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/OMERO.imagej-5.5.6/*

The following section shows
 * how to load a macro file linked to a dataset
 * how to analyze all the images in the dataset using the macro
 

In [2]:
g = new EasyForm("Enter credentials to connect")
g.addTextField("Server").onInit({g['Server'] = "outreach.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g


In [4]:
ga = new EasyForm("Select the Dataset to analyze")
ga.addTextField("DatasetID")
ga

In [14]:
import java.io.File
import java.io.FileOutputStream
import omero.gateway.Gateway
import omero.gateway.LoginCredentials
import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility
import omero.gateway.facility.MetadataFacility
import omero.gateway.model.DatasetData
import omero.gateway.model.FileAnnotationData
import omero.log.SimpleLogger
import omero.model.OriginalFile

import ij.IJ

//Connection information
HOST = g['Server']
USERNAME = g['UserName']
PASSWORD = g['Password']

PORT = 4064
//size of the chunk to load
INC = 262144
//Dataset to analyze Convert the String as long
dataset_id = ga['DatasetID'].toLong()


// Helper methods

def connect_to_omero() {
    "Connect to OMERO"

    credentials = new LoginCredentials()
    credentials.getServer().setHostname(HOST)
    credentials.getServer().setPort(PORT)
    credentials.getUser().setUsername(USERNAME.trim())
    credentials.getUser().setPassword(PASSWORD.trim())
    simpleLogger = new SimpleLogger()
    gateway = new Gateway(simpleLogger)
    gateway.connect(credentials)
    return gateway

}

def get_images(gateway, dataset_id) {
    "List all image's ids contained in a Dataset"
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    browse = gateway.getFacility(BrowseFacility)
    ids = new ArrayList(1)
    ids.add(new Long(dataset_id))
    return browse.getImagesForDatasets(ctx, ids)
}

def open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(HOST)
    options.append("\nuser=")
    options.append(USERNAME.trim())
    options.append("\nport=")
    options.append(PORT)
    options.append("\npass=")
    options.append(PASSWORD.trim())
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

// Load the macro file
def get_macro_file(gateway, dataset_id) {
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    svc = gateway.getFacility(MetadataFacility)
    //Find the dataset
    browse = gateway.getFacility(BrowseFacility)
    object = browse.findIObject(ctx, "omero.model.Dataset", dataset_id)
    dataset = new DatasetData(object);
    users = new ArrayList(1)
    users.add(new Long(user.getId()))
    types = new ArrayList(1)
    types.add(FileAnnotationData.class)
    annotations = svc.getAnnotations(ctx, dataset, types, users)
    
    store = gateway.getRawFileService(ctx);
    querySvc = gateway.getQueryService(ctx)
    index = 0
    annotations.each { fa ->
        of = (OriginalFile) querySvc.get("omero.model.OriginalFile", fa.getFileID());
        name = of.getName().getValue()
        //load the first annotation with ijm
        if (name.endsWith(".ijm") && index == 0) {
            file = File.createTempFile(name, ".ijm")
            stream = new FileOutputStream(file)
            store.setFileId(fa.getFileID());
            offset = 0;
            size = of.getSize().getValue();
            try {
                for (offset = 0; (offset+INC) < size;) {
                    stream.write(store.read(offset, INC));
                    offset += INC;
                }
            } finally {
                stream.write(store.read(offset, (int) (size-offset)));
                stream.close();
            }
            index++
        }
    }
    store.close()
    return file
}

// Main part of the script

println "connecting..."
gateway = connect_to_omero()
exp = gateway.getLoggedInUser()
group_id = exp.getGroupId()
exp_id = exp.getId()

// Load all the images contained in the dataset
println "loading images..."
images = get_images(gateway, dataset_id)

// Load the file annotations linked to the dataset.
println "loading macro..."
macro_file = get_macro_file(gateway, dataset_id)

//Apply the macro to all the images in the dataset
println "analyzing images..."
images.each { img ->
    // if target_user ~= None:
    // Switch context to target user and open omeroImage as ImagePlus object
    img_id = img.getId()
    println "Running macro on image: "+img_id
    imp = open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, String.valueOf(img_id))

    imp = IJ.getImage()
    
    IJ.runMacroFile(macro_file.getAbsolutePath())
    //TODO save output
    imp.changes = false     /// Prevent "Save Changes?" dialog
    imp.close()
}
// Close the connection
gateway.disconnect()
//Delete macrofile
macro_file.delete()
println "processing done"

connecting...
loading images...
loading macro...
analyzing images...
Running macro on image: 55206
processing done


null